In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

template = """
A metodologia **RASE N1** transforma textos em unidades menores, onde cada unidade contém **apenas uma única regra computável** com métricas claras.  

### **Instruções:**  
1. **Divida o texto** em sentenças curtas e diretas, respeitando a metodologia **RASE N1**.  
2. **Cada sentença deve conter somente uma única regra computável**.  
3. **Não remova nenhum dos seguintes elementos:**  
   - **Aplicabilidade:** Onde ou quando a regra se aplica.  
   - **Seleção:** Elemento específico dentro da aplicabilidade.  
   - **Requisito:** O que deve ser feito.  
   - **Exceção:** Casos que não precisam seguir a regra.  
4. **A resposta deve conter apenas os textos reformulados, sem explicações ou títulos.**  
5. **Cada frase deve ser separada por `\n`, garantindo uma quebra de linha entre elas.**  
6. **Todas as frases devem ser convertidas em afirmações lógicas.**  

### **Exemplo 1:**  

#### **Entrada:**  
"A inclinação transversal da superfície deve ser de até 2 % para pisos internos e de até 3 % para pisos externos. A inclinação longitudinal da superfície deve ser inferior a 5 %. Inclinações iguais ou superiores a 5 % são consideradas rampas e, portanto, devem atender a 6.6."  

#### **Saída:**  
Pisos internos devem ter inclinação transversal de no máximo 2%.\n  
Pisos externos devem ter inclinação transversal de no máximo 3%.\n  
A inclinação longitudinal da superfície deve ser inferior a 5%.\n  
Inclinações iguais ou superiores a 5% são consideradas rampas e devem atender à norma 6.6.\n  

### **Exemplo 2:**  

#### **Entrada:**  
"Os acessos devem ser vinculados através de rota acessível à circulação principal e às circulações de emergência. Os acessos devem permanecer livres de quaisquer obstáculos de forma permanente."  

#### **Saída:**  
Os acessos devem ser vinculados através de rota acessível à circulação principal e às circulações de emergência.\n  
Os acessos devem permanecer livres de quaisquer obstáculos de forma permanente.\n  

### **Agora, transforme o texto abaixo utilizando a metodologia RASE N1:**  

#### **Texto:**  
{text}  

#### **Resposta:**    
"""

model = OllamaLLM(model="splitpierre/bode-alpaca-pt-br")
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

result = chain.invoke({"text": "Nas edificações e equipamentos urbanos, todas as entradas, bem como as rotas de interligação às funções do edifício, devem ser acessíveis."})
print(result)
print("----------------------------------------------------------------------")
result = chain.invoke({"text": "Os acessos devem ser vinculados através de rota acessível à circulação principal e às circulações de emergência. Os acessos devem permanecer livres de quaisquer obstáculos de forma permanente."})
print(result.replace(". ", ".\n"))

As entradas devem estar acessíveis.
----------------------------------------------------------------------
Os acessos devem ser vinculados através da rota acessível à circulação principal e às circulações de emergência.
Os acessos devem permanecer livres de qualquer obstáculo em forma permanente.
